<a href="https://colab.research.google.com/github/akalend/mobile_nlp_analisys/blob/master/Text_Classification_With_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 1.4MB 19.0MB/s 
     |████████████████████████████████| 2.9MB 51.8MB/s 
     |████████████████████████████████| 890kB 39.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=cff4afb3913e4a3996c6b1bf24ad4c30980e273648b024e27fc370b6d4afc6c6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('russian')) 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
import pandas as pd


f = open('data/review.csv', 'r')
in_text = []
line = True
while line:
  line = f.readline()
  if not line:
    break
  in_text.append(line)




In [52]:
df = pd.DataFrame(in_text, columns=['comment'])

In [53]:
df

,comment
0,На девятый день рождения дочка попросила новый...
1,Привет всем друзьям и посетителям Отзовика!Сма...
2,За 10 000 рублей данный смартфон это оптимальн...
3,"Привет, дорогой читатель(а может и пользовател..."
4,"Искал себе телефон небольших размеров, чтоб уд..."
...,...
2584,"Покупала телефон за 7500, сейчас он вышел из п..."
2585,Покупал его новый за 4300 рублей. В нём мне оч...
2586,Пользуюсь смартфоном Fly IQ 441 уже больше год...
2587,Всем привет. Этот телефон мне очень понравился...


In [54]:
df.drop_duplicates(subset=['comment'],inplace=True)

In [57]:
stop_words

{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [66]:
c_text=[]
jj = 0
for i in df['comment']:
  j = ' '.join([ w for w in [w for w in i.split() if not w in stop_words]])
  c_text.append(j)
df['comment']=c_text

In [75]:
df.insert(0, "Prz", 1)

In [76]:
df

,Prz,comment
0,1,На девятый день рождения дочка попросила новый...
1,1,Привет всем друзьям посетителям Отзовика!Смарт...
2,1,За 10 000 рублей данный смартфон это оптимальн...
3,1,"Привет, дорогой читатель(а пользователь) серви..."
4,1,"Искал телефон небольших размеров, удобно одно ..."
...,...,...
2584,1,"Покупала телефон 7500, вышел продажи. Достоинс..."
2585,1,Покупал новый 4300 рублей. В нём очень понрави...
2586,1,Пользуюсь смартфоном Fly IQ 441 года хочу сказ...
2587,1,Всем привет. Этот телефон очень понравился сво...


In [79]:
df['Prz'].value_counts()

1    2519
Name: Prz, dtype: int64

In [81]:
possible_labels = df.Prz.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{1: 0}

In [83]:
df['label'] = df.Prz

In [84]:
df.head()

,Prz,comment,label
0,1,На девятый день рождения дочка попросила новый...,1
1,1,Привет всем друзьям посетителям Отзовика!Смарт...,1
2,1,За 10 000 рублей данный смартфон это оптимальн...,1
3,1,"Привет, дорогой читатель(а пользователь) серви...",1
4,1,"Искал телефон небольших размеров, удобно одно ...",1


In [92]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [93]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [94]:
df.groupby(['Prz', 'label', 'data_type']).count()

comment
Prz label data_type         
1   1     train         2141
          val            378

In [95]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [99]:
 df[df.data_type=='train'].comment.values

array(['На девятый день рождения дочка попросила новый телефон. Я сама стала задумываться том, купить современный (чем её старенький Alcatel 4034D) аппарат неё. Технику покупаю крайне редко особо разбираюсь. Поспрашивала знакомых какие недорогие варианты покупают (чтобы районе 10000). Как оказалось, основном берут Huawei Honor. Как понимаю производитель один. Или вычитала интернете Honor - это отдельный бред компании Huawei. В итоге решила доехать магазина "DNS" месте подобрать.Конечно телефону это имеет никакого отношения, просто хочется отметить. Обслуживали молча. Не посоветовать, уточнить, предложить. Вообще выражению лица понятно, интереса покупатели никакого представляем. Постояли, потоптались, ткнули пару моделей пальцем (ну умею выбирать). Остановились смартфоне Honor 8a prime стоимостью 9999 рублей.Вот чек покупки.С момента покупки написания отзыва прошло месяца. Хочу поделиться своими впечатлениями.Домой принесли такую картонную прямоугольную коробочку.Информация ней.-Из муку

In [100]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].comment.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [102]:


encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].comment.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values,dtype=torch.long)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values,dtype=torch.long)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [103]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [104]:
len(dataset_train), len(dataset_val)

(2141, 378)

In [105]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [106]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

In [107]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [108]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [110]:
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [111]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [112]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [113]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [114]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [123]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

RuntimeError: ignored

In [118]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to('cuda')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [122]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))

FileNotFoundError: ignored

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
predictions

array([[ 6.701269  ,  1.265477  , -1.0989971 , -2.1158788 , -2.30888   ,
        -2.0789335 ],
       [ 6.775178  , -0.08133557, -1.3402019 , -1.8453351 , -1.5773239 ,
        -1.409198  ],
       [ 6.8711667 ,  0.27579504, -1.3199737 , -1.9986652 , -1.7535454 ,
        -1.5519205 ],
       ...,
       [-2.530412  , -2.0244465 , -1.1328572 ,  0.34031317,  2.2881658 ,
         3.544518  ],
       [ 0.9167181 ,  2.4891872 ,  2.6774354 , -0.08066647, -2.762556  ,
        -4.28428   ],
       [-1.1291463 ,  2.0318134 ,  2.7955718 ,  1.5517545 , -2.327366  ,
        -3.8731647 ]], dtype=float32)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0.0
Accuracy: 2208/2449

Class: 4.0
Accuracy: 571/1002

Class: 1.0
Accuracy: 197/611

Class: 3.0
Accuracy: 495/768

Class: 2.0
Accuracy: 122/289

Class: 5.0
Accuracy: 28/68



In [ ]:
def predicti(dataloader_test):

    model.eval()
    
    loss_val_total = 0
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
        # loss = outputs[0]
        logits = outputs[0]
        # loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        # label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        # true_vals.append(label_ids)
    
    # loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    # true_vals = np.concatenate(true_vals, axis=0)
            
    return predictions

In [ ]:
import csv
def read_test(path="test.csv"):
    texts = []
    # stop_words = set(stopwords.words('english')) 
    # lemmatizer = WordNetLemmatizer()
    # uuids = []
    with open(path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', )
        # next(reader)
        for row in reader:
            text = row
            text = ' '.join([lemmatizer.lemmatize(w) for w in [w for w in text[0].split() if not w in stop_words]])
            # uuids.append(uuid)
            texts.append(text)
    return (texts)

In [121]:
import numpy as np


In [120]:
test=read_test("dataset_316011_2.txt")
test=np.ravel(test)

NameError: ignored

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    test, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

dataset_test = TensorDataset(input_ids_test, attention_masks_test)

In [ ]:
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:
device='cuda'

In [ ]:
predictions = predicti(dataloader_test)

In [ ]:
x=np.argmax(predictions, axis=1).flatten()

In [ ]:
x

array([0, 1, 0, ..., 3, 0, 3])

In [ ]:
np.savetxt('test.txt', x, delimiter=',',fmt='%s')   # X is an array